In [ ]:
!conda activate PythonData

/bin/bash: line 1: conda: command not found


In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler,OneHotEncoder
#from keras.metrics import categorical_accuracy
#import tensorflow as tf
import glob
import os


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# merge the files
disease_path = "/content/drive/My Drive/Colab Notebooks/Disease_symptom_and_patient_profile_dataset.csv"

# Concatinate the files
disease_df = pd.read_csv(disease_path,encoding="utf-8")

# Review the DataFrame
disease_df

,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,Influenza,Yes,No,Yes,Yes,19,Female,Low,Normal,Positive
1,Common Cold,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
2,Eczema,No,Yes,Yes,No,25,Female,Normal,Normal,Negative
3,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
4,Asthma,Yes,Yes,No,Yes,25,Male,Normal,Normal,Positive
...,...,...,...,...,...,...,...,...,...,...
344,Stroke,Yes,No,Yes,No,80,Female,High,High,Positive
345,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
346,Stroke,Yes,No,Yes,No,85,Male,High,High,Positive
347,Stroke,Yes,No,Yes,No,90,Female,High,High,Positive


In [ ]:
#identify the data types in the file before transforming
disease_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-null    object
 1   Fever                 349 non-null    object
 2   Cough                 349 non-null    object
 3   Fatigue               349 non-null    object
 4   Difficulty Breathing  349 non-null    object
 5   Age                   349 non-null    int64 
 6   Gender                349 non-null    object
 7   Blood Pressure        349 non-null    object
 8   Cholesterol Level     349 non-null    object
 9   Outcome Variable      349 non-null    object
dtypes: int64(1), object(9)
memory usage: 27.4+ KB


In [ ]:
disease_df.nunique()

Disease                 116
Fever                     2
Cough                     2
Fatigue                   2
Difficulty Breathing      2
Age                      26
Gender                    2
Blood Pressure            3
Cholesterol Level         3
Outcome Variable          2
dtype: int64

In [ ]:
#initiate scaling
disease_data_scaled = StandardScaler().fit_transform(disease_df[["Age"]])
#create df with scaled data
disease_scaled_df = pd.DataFrame(disease_data_scaled,columns=["Age"])
#show the newly scaled df
disease_scaled_df.head()

,Age
0,-2.091160
1,-1.631964
2,-1.631964
3,-1.631964
4,-1.631964


In [ ]:
label_encoder = preprocessing.LabelEncoder()

disease_df["Disease"]=label_encoder.fit_transform(disease_df["Disease"])
disease_df['Gender'] = label_encoder.fit_transform(disease_df['Gender'])
disease_df['Difficulty Breathing'] = label_encoder.fit_transform(disease_df['Difficulty Breathing'])
disease_df['Fever'] = label_encoder.fit_transform(disease_df['Fever'])
disease_df['Cough'] = label_encoder.fit_transform(disease_df['Cough'])
disease_df['Fatigue'] = label_encoder.fit_transform(disease_df['Fatigue'])
disease_df['Blood Pressure'] = label_encoder.fit_transform(disease_df['Blood Pressure'])
disease_df['Outcome Variable'] = label_encoder.fit_transform(disease_df['Outcome Variable'])
disease_df['Cholesterol Level'] = label_encoder.fit_transform(disease_df['Cholesterol Level'])

In [ ]:
disease_concat_df = pd.concat([disease_df,disease_scaled_df],axis=1)
disease_concat_df.shape

(349, 11)

In [ ]:
y = disease_concat_df["Outcome Variable"].values
X = disease_concat_df.drop(["Outcome Variable","Age","Fatigue"],1).values
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

<ipython-input-39-929dba2fbb01>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = disease_concat_df.drop(["Outcome Variable","Age","Fatigue"],1).values


In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(solver='newton-cholesky', random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [ ]:
# Make a prediction using the testing data
test_predictions = logistic_regression_model.predict(X_test)
pd.DataFrame({'Predictions': test_predictions, 'Actual': y_test})

,Predictions,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,0,1
...,...,...
83,0,0
84,1,1
85,0,0
86,1,1


In [ ]:
# Print the balanced_accuracy score of the model
print(f"The balanced accuracy score of the model is: {balanced_accuracy_score(y_test, test_predictions)}")

The balanced accuracy score of the model is: 0.5763157894736842


In [ ]:
# Generate a confusion matrix for the model
cf_test_matrix = confusion_matrix(y_test, test_predictions)
cf_test_matrix

array([[21, 17],
       [20, 30]])

In [ ]:
# Print the classification report for the model
testing_report = classification_report(y_test, test_predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       0.51      0.55      0.53        38
           1       0.64      0.60      0.62        50

    accuracy                           0.58        88
   macro avg       0.58      0.58      0.58        88
weighted avg       0.58      0.58      0.58        88

